2. Model Initilization

In [ ]:


# build 
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, input_dim=X.shape[1], activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(len(cats), activation='softmax')
    ])

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

0. Set Hyperparamters

In [ ]:
test_ratio = 0.7     # This is a placegholder Value

1. Import Data

In [ ]:
# 1. Import data
NY_data = pd.read_csv("data/NY_data.csv")

# 2. Break data into x and y
# 2. Split data into Train vs. Test
# ALter values as needed
X_test, X_val, y_test, y_val = train_test_split(X, y, test_size=test_ratio, random_state=0, shuffle=True, stratify=y)

2. Model Initilization

In [ ]:


# build 
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(6, input_dim=X.shape[1], activation = 'relu'),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(len(cats), activation='softmax')
    ])

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

0. Set Hyperparamters

In [5]:
test_ratio = 0.7     # This is a placegholder Value

1. Import Data

In [1]:
# 1. Import data
NY_data = pd.read_csv("data/NY_data.csv")
# 2. Split data into Train vs. Test
# ALter values as needed
X_test, X_val, y_test, y_val = train_test_split(X, y, test_size=test_ratio, random_state=0, shuffle=True, stratify=y)